# 垃圾邮件分类
> 需要完成一个二分类问题，0为正常，1为垃圾邮件

In [1]:
!conda env list

# conda environments:
#
base                  *  /opt/conda
federated_learning       /opt/conda/envs/federated_learning
python2                  /opt/conda/envs/python2
pytorch1.8               /opt/conda/envs/pytorch1.8
rapids                   /opt/conda/envs/rapids
tf1.14                   /opt/conda/envs/tf1.14
tf2.4.1                  /opt/conda/envs/tf2.4.1



## 1. 导入依赖

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, classification_report
import wandb

/root/.local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-05-04 09:32:54.096635: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-05-04 09:32:54.096736: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-05-04 09:32:54.096746: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU w

## 2. 初始化wandb帮助记录log, 设置好device, 定义模型结构
定义模型结构，继承nn.Module，重写forward方法
bert-base-chinese得到last_hidden_state,然后自己做一个linear作为classification的head

In [3]:
wandb.init(project="text_cls", entity='cocoshe')
# wandb.config = {
#     'batch_size': 32,
#     'epochs': 5,
#     'lr': 5e-5,
#     'max_seq_length': 128,
# }
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class bert_cls(nn.Module):
    def __init__(self, model_name, num_labels, freeze_bert=True):
        super(bert_cls, self).__init__()
        self.bert = BertModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, num_labels)
        self.softmax = nn.Softmax(dim=-1)

        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False

    def forward(self, input_ids, attention_mask):
        output = self.bert(input_ids, attention_mask)
        hidden_state = output['last_hidden_state'][:, 0, :]
        hidden_state = self.dropout(hidden_state)
        logits = self.classifier(hidden_state)
        return logits

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: cocoshe (use `wandb login --relogin` to force relogin)
2022-05-04 09:33:26.068727: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-05-04 09:33:26.068823: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-05-04 09:33:26.068930: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, pleas

## 3. 定义dataset类，继承pytorch的Dataset类，重写__getitem__和__len__方法，继承__init__方法

In [4]:
class MyDataset(Dataset):
    def __init__(self, df, model_name='bert-base-chinese'):
        self.data_df = df
        self.tokenizer = BertTokenizer.from_pretrained(model_name)

    def __len__(self):
        return len(self.data_df)

    def __getitem__(self, idx):
        input_text = self.data_df.iloc[idx, 1]
        label = self.data_df.iloc[idx, 0]

        tokenizer = self.tokenizer
        token = tokenizer(input_text, padding='max_length', truncation=True, max_length=128, return_tensors='pt', return_token_type_ids=False)
        input_ids = token['input_ids'].squeeze(0)
        attention_mask = token['attention_mask'].squeeze(0)

        return input_ids.to(device), attention_mask.to(device), label

## 4. 拿到配套的tokenizer，实例化model并放到device上，实例化损失函数和优化器，设置一些超参数
> 模型只做linear probing，默认把bert部分freeze掉

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = bert_cls('bert-base-chinese', num_labels=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
epochs = 5

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 5. 定义数据集切分，训练集和验证集
> ps:也可以用sklearn的train_test_split

In [6]:
def train_test_split(input_df, test_size=0.2, shuffle=True, random_state=None):
    if shuffle:
        input_df = input_df.sample(frac=1)
    ret_train_df = input_df[:int(len(input_df) * (1 - test_size))]
    ret_test_df = input_df[int(len(input_df) * (1 - test_size)):]
    return ret_train_df, ret_test_df


## 6. 导入数据集，做7：3切分，然后生成train_loader和test_loader

In [7]:
# split train and test data directly(no need to split input and label, that's why not use sklearn.model_selection.train_test_split)
df = pd.read_csv('dataset/message80W.csv', header=None, index_col=0, names=['label', 'text'])
train_df, test_df = train_test_split(df, test_size=0.3, shuffle=True)

train_dataset = MyDataset(train_df, model_name='bert-base-chinese')
test_dataset = MyDataset(test_df, model_name='bert-base-chinese')

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


## 7. trian

In [8]:
# train
# def train(epochs, model, train_loader, optimizer, criterion):
model.train()
for epoch in range(epochs):
    for i, batch in enumerate(tqdm(train_loader)):
        input_ids, attention_mask, label = batch
        logits = model(input_ids, attention_mask)
        loss = criterion(logits, label.to(device))
        # print('loss: ', loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        wandb.log({'epoch': epoch,'step': i, 'loss': loss.item()})


100%|██████████| 17500/17500 [25:20<00:00, 11.51it/s]


## 8. evaluate

In [38]:
# evaluate
# def eval_model(epoch, model, test_loader):
global best_f1
with torch.no_grad():
    model.eval()
    eval_steps = 0
    pred_list = []
    label_list = []
    for i, batch in enumerate(tqdm(test_loader)):
        input_ids, attention_mask, label = batch
        logits = model(input_ids, attention_mask)
        pred_list += (torch.argmax(logits, dim=-1))
        label_list += label
        eval_steps += 1

  

Error in callback <function _WandbInit._resume_backend at 0x7fb2f341f710> (for pre_run_cell):


Exception: The wandb backend process has shutdown

100%|██████████| 7500/7500 [10:17<00:00, 12.15it/s]


AttributeError: 'list' object has no attribute 'cpu'

Error in callback <function _WandbInit._pause_backend at 0x7fb2f341f7a0> (for post_run_cell):


Exception: The wandb backend process has shutdown

In [45]:
# label_list, pred_list = [item.cpu().numpy() for item in label_list], [item.cpu().numpy() for item in pred_list]
f1 = f1_score(label_list, pred_list, average='macro')
acc = accuracy_score(label_list, pred_list)
precision = precision_score(label_list, pred_list, average='macro')
recall = recall_score(label_list, pred_list, average='macro')

# wandb.log({'epoch': epoch, 'f1': f1, 'acc': acc, 'precision': precision, 'recall': recall})


torch.save(model.state_dict(), 'model/bert_cls_model.pth')
report = classification_report(label_list, pred_list)
print('classification_report: \n', report)

Error in callback <function _WandbInit._resume_backend at 0x7fb2f341f710> (for pre_run_cell):


Exception: The wandb backend process has shutdown

classification_report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    216083
           1       0.99      0.97      0.98     23917

    accuracy                           1.00    240000
   macro avg       0.99      0.99      0.99    240000
weighted avg       1.00      1.00      1.00    240000

Error in callback <function _WandbInit._pause_backend at 0x7fb2f341f7a0> (for post_run_cell):


Exception: The wandb backend process has shutdown

## 9. 分类报告

最终的分类报告如下:


![cls_report](../static/cls_report.png)

